# An example of iteration over DataFrame

Firstly create a virtual environment and install pandas httpx and codetiming

```bash
python3 -m venv .venv
.venv/bin/active
python -m pip install pandas httpx codetiming
```

In [9]:
import pandas as pd
websites = pd.read_csv("popular_websites.csv")
websites

,name,url,total_views
0,Google,https://www.google.com,5.207268e+11
1,YouTube,https://www.youtube.com,2.358132e+11
2,Facebook,https://www.facebook.com,2.230157e+11
3,Yahoo,https://www.yahoo.com,1.256544e+11
4,Wikipedia,https://www.wikipedia.org,4.467364e+10
5,Baidu,https://www.baidu.com,4.409759e+10
6,Twitter,https://twitter.com,3.098676e+10
7,Yandex,https://yandex.com,2.857980e+10
8,Instagram,https://www.instagram.com,2.621520e+10
9,AOL,https://www.aol.com,2.321232e+10


## Iterating over pandas
Assuming you want to check website connectivity lets define a function and then iterate over DataFrame

In [10]:
import httpx
def check_connection(name, url):
    try:
        response = httpx.get(url)
        location = response.headers.get("location")
        if location is None or location.startswith(url):
            print(f"{name} is online!")
        else:
            print(f"{name} is online! But redirects to {location}")
        return True
    except httpx.ConnectError:
        print(f"Failed to establish a connection with {url}")
### Lopping over websites DataFrame
for web in websites.itertuples():
    check_connection(web.name,web.url)


Google is online!
YouTube is online!
Facebook is online!
Yahoo is online!
Wikipedia is online!
Baidu is online!
Twitter is online!
Yandex is online!
Instagram is online!
AOL is online!
Netscape is online! But redirects to https://www.aol.com/
Failed to establish a connection with https://alwaysfails.example.com


## Using vectorized methods over iteration for Summing a Column

### Iterating over DataFrame

In [11]:
def loop_sum(websites):
    total = 0
    for web in websites.itertuples():
        total += web.total_views
    return total
print('Total using loop_sum: {}'.format(loop_sum(websites)))

Total using loop_sum: 1302981160000.0


### List Comprenhension

In [12]:
def python_sum(websites):
    return sum(web.total_views for web in websites.itertuples())
print('Total using Python List Comprehension: {}'.format(python_sum(websites)))

Total using Python List Comprehension: 1302981160000.0


### Using pandas

In [14]:
def pandas_sum(websites):
    return websites["total_views"].sum()
print('Total using Pandas Sum: {}'.format(pandas_sum(websites)))

Total using Pandas Sum: 1302981160000.0


## Checking out execution time for each method above

### Checking out with original data source

In [19]:
from codetiming import Timer
for func in [loop_sum, python_sum, pandas_sum]:
    with Timer(name=func.__name__, text="{name:20}: {milliseconds: .2f} ms"):
        func(websites)

loop_sum            :  1.14 ms
python_sum          :  0.83 ms
pandas_sum          :  0.31 ms


There is a significant performance between each method

### Checking out with thousands of web sites

In [21]:
from codetiming import Timer
websites1000 = pd.concat(1000*[websites])
for func in [loop_sum, python_sum, pandas_sum]:
    with Timer(name=func.__name__, text="{name:20}: {milliseconds: .2f} ms"):
        func(websites1000)

loop_sum            :  15.76 ms
python_sum          :  10.18 ms
pandas_sum          :  0.36 ms


As long as the DataFrame is increasing the pandas methos is more efficiant

## Using vectorized methods over iteration with an Intermediate Columns

In [22]:
import pandas as pd
products = pd.read_csv("products.csv")
products

,month,sales,unit_price
0,january,3,0.50
1,february,2,0.53
2,march,5,0.55
3,april,10,0.71
4,may,8,0.66


### Iterating method with and intermedia compute

In [25]:
import pandas as pd
products = pd.read_csv("products.csv")
def loop_cumsum(products):
    comulative_sum = []
    for prod in products.itertuples():
        income = prod.sales * prod.unit_price
        if comulative_sum:
            comulative_sum.append(comulative_sum[-1] + income)
        else:
            comulative_sum.append(income)
    return products.assign(comulative_income=comulative_sum)

loop_cumsum(products)


,month,sales,unit_price,comulative_income
0,january,3,0.50,1.50
1,february,2,0.53,2.56
2,march,5,0.55,5.31
3,april,10,0.71,12.41
4,may,8,0.66,17.69


### Pandas Comulative sum using intermedia column

In [27]:
import pandas as pd
products = pd.read_csv("products.csv")

def pandas_cumsum(products):
    return products.assign(
        income = lambda df: df["sales"] * df["unit_price"],
        comulative_income= lambda df: df["income"].cumsum()
    ).drop(columns="income")

pandas_cumsum(products)

,month,sales,unit_price,comulative_income
0,january,3,0.50,1.50
1,february,2,0.53,2.56
2,march,5,0.55,5.31
3,april,10,0.71,12.41
4,may,8,0.66,17.69


## Checking out Performance with intermedia column

### Checking out with original data

In [29]:
from codetiming import Timer
import pandas as pd

for func in [loop_cumsum, pandas_cumsum]:
    prod = pd.read_csv("products.csv")
    with Timer(name=func.__name__, text="{name:20}: {milliseconds: .2f} ms"):
        func(prod)


loop_cumsum         :  1.42 ms
pandas_cumsum       :  2.62 ms


Apparently the for method looks faster and indeed it's faster for tiny dataset.

### Checking out performance with thousands of rows  

In [32]:
from codetiming import Timer
import pandas as pd

for func in [loop_cumsum, pandas_cumsum]:
    prod = pd.read_csv("products.csv")
    prod = pd.concat(1000*[prod])
    with Timer(name=func.__name__, text="{name:20}: {milliseconds: .2f} ms"):
        func(prod)

loop_cumsum         :  7.60 ms
pandas_cumsum       :  2.54 ms


We can see as long as the datase is increasing the vectorised method performance is better that iteration one.

#### Caveats
* All test has been done in a MacBook Pro Mid 2012 with 16GB RAM and Ubuntu 20.04.1
* Based on [How to Iterate Over Rows in pandas, and Why You Shouldn't](https://realpython.com/pandas-iterate-over-rows/)